In [1]:
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

hide_toggle()

***Подгружаем необходимые инструменты:***

In [2]:
import warnings

warnings.filterwarnings('ignore')

hide_toggle()

In [3]:
import numpy as np
import pandas as pd
import gzip
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from collections import Counter
import string

import nltk
from nltk import *
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import re

from gensim import models, corpora

#import polyglot
#from polyglot.text import Text as T
nltk.download()
hide_toggle()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


***Загружаем текстовые данные:***

In [4]:
path = 'qa_Appliances.json.gz'


def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('qa_Appliances.json.gz')

#data_answers = df['answer']
data_answers = df['answer'].apply(lambda x: x.lower())
#data_answers = str(data_answers)
data_answers

hide_toggle()

In [5]:
data_answers.head(5)

0     i replaced my old one with this without a hitch.
1    this may help insinkerator model badger-1: bad...
2    plumbing connections will vary with different ...
3    it does not come with a power cord. it does co...
4    check if you dropped something inside.usually ...
Name: answer, dtype: object

***Функции по очистке данных (освобождение от цифр и знаков пунктуации):***

In [6]:
#cleaning and processing

#cleaning from punctuation
def replace_punctuation(x):  # From S.Lott's solution
    for c in string.punctuation:
        x=x.replace(c,"")
    return x

#removing numbers
def clean_numbers(x):
    x = str(x)
    re.sub(r'\d+', '', x)
    return x

hide_toggle()

***Вычисляем частотность слов (N самых употребляемых слов):***

In [7]:

#top N words     всего текста
def top_N_words(N):
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf.fit(data_answers)
    X = tfidf.transform(data_answers)
    idx = np.ravel(X.sum(axis=0).argsort(axis=1))[::-1][:N]
    top_n_words = np.array(tfidf.get_feature_names())[idx].tolist()
    return top_n_words


hide_toggle()

In [8]:
top_N_words(10)

['yes',
 'model',
 'fit',
 'don',
 'sure',
 'does',
 'know',
 'just',
 'work',
 'filter']

***Вычисление эмоциональной окрашенности текста:***

In [9]:
#sentiment analysis
#for english version we'll use vader sentiment analyzer

def emotions(data_answers):
    data_answers = str(data_answers)
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    sid = SentimentIntensityAnalyzer()
    sentences = tokenize.sent_tokenize(data_answers)
    scores = dict([('pos', 0), ('neu', 0), ('neg', 0), ('compound', 0)])
    for sentence in sentences:
        ss = sid.polarity_scores(sentence)
        for k in sorted(ss):
            scores[k] += ss[k]

    return scores 


hide_toggle()

In [10]:
emotions(data_answers)

{'compound': 6.4398,
 'neg': 0.21400000000000002,
 'neu': 20.912999999999997,
 'pos': 2.8729999999999998}

***Вычленение главных тем:***      (тест)

***вариант-1***

In [11]:
#LDA gensim version-2
import re
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords
 
NUM_TOPICS = 10
STOPWORDS = stopwords.words('english')
texts = str(data_answers)
#texts = text.split()
    
def clean_text(texts):
    tokenized_text = word_tokenize(texts.lower())
    cleaned_text = [t for t in tokenized_text if t not in STOPWORDS and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
    return cleaned_text
 
# For gensim we need to tokenize the data and filter out stopwords
tokenized_data = []
for text in texts:
    tokenized_data.append(clean_text(texts))
 
 
# Build a Dictionary - association word to numeric id
dictionary = corpora.Dictionary(tokenized_data)
 
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
 
# Have a look at how the 20th document looks like: [(word_id, count), ...]
print(corpus[20])
# [(12, 3), (14, 1), (21, 1), (25, 5), (30, 2), (31, 5), (33, 1), (42, 1), (43, 2),  ...
 
# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 
# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)

hide_toggle()

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 4), (16, 2), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 3), (46, 1), (47, 1), (48, 1), (49, 1), (50, 7), (51, 1), (52, 1), (53, 2), (54, 1), (55, 1), (56, 3), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 4), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 2), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 2), (89, 1), (90, 2), (91, 1), (92, 1), (93, 1), (94, 3), (95, 1), (96, 1), (97, 1), (98, 3), (99, 2), (100, 3), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 2), (109, 1), (110, 1),

***вариант-2***

In [12]:
#LDA scikit-learn version-2

from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data_answers)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Non-Negative Matrix Factorization Model
nmf_model = NMF(n_components=NUM_TOPICS)
nmf_Z = nmf_model.fit_transform(data_vectorized)
print(nmf_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Latent Semantic Indexing Model
lsi_model = TruncatedSVD(n_components=NUM_TOPICS)
lsi_Z = lsi_model.fit_transform(data_vectorized)
print(lsi_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 

#Let's see how the first document in the corpus looks like in different topic spaces
print(lda_Z[0])
print(nmf_Z[0])
print(lsi_Z[0])


def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

print("LDA Model:")
print_topics(lda_model, vectorizer)
print("=" * 20)
 
print("NMF Model:")
print_topics(nmf_model, vectorizer)
print("=" * 20)
 
print("LSI Model:")
print_topics(lsi_model, vectorizer)
print("=" * 20)

hide_toggle()

(9011, 10)
(9011, 10)
(9011, 10)
[0.03333333 0.03333333 0.03333333 0.03334444 0.69994964 0.03335798
 0.03333333 0.03333333 0.03333333 0.03334794]
[0.         0.         0.00237005 0.00025971 0.00204277 0.01964319
 0.         0.         0.         0.        ]
[ 0.0866091   0.05006994 -0.01129209 -0.03681695 -0.01664433 -0.04965268
 -0.07266082  0.0170258  -0.0695201  -0.03787401]
LDA Model:
Topic 0:
[('model', 997.7951671007862), ('number', 747.0913698758258), ('filter', 498.2533905506248), ('com', 388.2502455942136), ('parts', 354.13901659897226), ('amazon', 324.29280430561744), ('whirlpool', 318.2895595274265), ('replacement', 294.4099181848643), ('filters', 265.9762482550687), ('refrigerator', 262.8548536610863)]
Topic 1:
[('yes', 1702.4767148803103), ('ice', 531.2340897867189), ('fit', 397.15796485247813), ('does', 317.7898182324066), ('maker', 189.8642871299094), ('thank', 180.23110456910396), ('work', 147.89588021576677), ('freezer', 147.41432611040668), ('valve', 142.924857323689

***Визуализация выделенных тем:***

In [13]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4      -86.194916   21.345306       1        1  15.163772
3      -40.026215  -56.767281       2        1  14.590369
0       18.221691  145.900635       3        1  12.211008
6      -76.592918  122.913803       4        1  11.008508
2       53.746300   63.929607       5        1  10.724398
8      -21.459627   69.859947       6        1   8.374993
1       11.142182    1.751809       7        1   7.815656
7      108.777611  117.437805       8        1   7.330770
9       62.919174  -56.841000       9        1   6.696516
5      116.381310   14.919238      10        1   6.084012, topic_info=     Category         Freq        Term        Total  loglift  logprob
2678  Default  1581.000000         yes  1581.000000  30.0000  30.0000
1528  Default  1073.000000       model  1073.000000  29.0000  29.0000
2585  Default  1107.000000       water  1107.000000  28.0000  28.0000
1597  Default   705.000000      number   705.000000  27.0000  27.0000
705   Default   806.000000         don   806.000000  26.0000  26.0000
1174  Default   494.000000         ice   494.000000  25.0000  25.0000
893   Default   919.000000      filter   919.000000  24.0000  24.0000
1300  Default   638.000000        know   638.000000  23.0000  23.0000
707   Default   377.000000        door   377.000000  22.0000  22.0000
1357  Default   576.000000        like   576.000000  21.0000  21.0000
2577  Default   469.000000      washer   469.000000  20.0000  20.0000
907   Default   785.000000         fit   785.000000  19.0000  19.0000
1195  Default   343.000000      inches   343.000000  18.0000  18.0000
1144  Default   379.000000        hope   379.000000  17.0000  17.0000
2341  Default   774.000000        sure   774.000000  16.0000  16.0000
698   Default   615.000000        does   615.000000  15.0000  15.0000
1109  Default   321.000000       helps   321.000000  14.0000  14.0000
1025  Default   509.000000        good   509.000000  13.0000  13.0000
1572  Default   685.000000        need   685.000000  12.0000  12.0000
443   Default   366.000000         com   366.000000  11.0000  11.0000
1873  Default   254.000000       range   254.000000  10.0000  10.0000
1688  Default   415.000000       parts   415.000000   9.0000   9.0000
2647  Default   660.000000        work   660.000000   8.0000   8.0000
1233  Default   242.000000   installed   242.000000   7.0000   7.0000
908   Default   257.000000        fits   257.000000   6.0000   6.0000
2495  Default   582.000000        unit   582.000000   5.0000   5.0000
1419  Default   357.000000     machine   357.000000   4.0000   4.0000
80    Default   403.000000      amazon   403.000000   3.0000   3.0000
245   Default   279.000000      bought   279.000000   2.0000   2.0000
1806  Default   291.000000     product   291.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
269   Topic10    39.839979      broken    40.710336   2.7779  -5.0729
2583  Topic10    38.393890       watch    39.264243   2.7771  -5.1099
335   Topic10    37.516787        cast    38.387002   2.7766  -5.1330
1042  Topic10    35.209715     griddle    36.079926   2.7751  -5.1964
9     Topic10    34.421714  absolutely    35.292108   2.7745  -5.2191
383   Topic10    32.908997       china    33.779271   2.7734  -5.2640
1768  Topic10    32.844795         pot    33.714988   2.7734  -5.2660
1761  Topic10    32.830387    position    33.705732   2.7732  -5.2664
432   Topic10    31.372981        coil    32.243228   2.7721  -5.3118
1713  Topic10    30.842262       phone    31.712711   2.7717  -5.3289
1158  Topic10    30.276026     housing    31.146308   2.7712  -5.3474
978   Topic10    29.806566    function    30.676865   2.7707  -5.3630
1336  Topic10    60.144250      length    64.559283   2.7287  -4.6610
293   Topic10   114.762704      burner   129.701672   2.6771  -4.0149
2073  Topic10    37.110381        seen   